In [5]:
import sqlite3
import pandas as pd

In [6]:
con = sqlite3.connect("/content/Chinook (Data Base File) (3).db")
con.text_factory = bytes

In [7]:
# define a helper function so the same syntax can be used
dbGetQuery = lambda con, q: pd.read_sql(q, con=con)

In [8]:
# 0. List all tables
print(dbGetQuery(con, "select name from sqlite_master where type = 'table';"))

                name
0           b'Album'
1          b'Artist'
2        b'Customer'
3        b'Employee'
4           b'Genre'
5         b'Invoice'
6     b'InvoiceLine'
7       b'MediaType'
8        b'Playlist'
9   b'PlaylistTrack'
10          b'Track'


In [9]:
# 1.	Display all of the information from the Track table sorted by length.
A = dbGetQuery(con, "select * from Track order by Milliseconds;")
print(A)

      TrackId                            Name  AlbumId  MediaTypeId  GenreId  \
0        2461  b'\xc9 Uma Partida De Futebol'      200            1        1   
1         168                   b'Now Sports'       18            1        4   
2         170                  b'A Statistic'       18            1        4   
3         178                        b'Oprah'       18            1        4   
4        3304                 b'Commercial 1'      258            1       17   
...       ...                             ...      ...          ...      ...   
3498     3227  b'Battlestar Galactica, Pt. 2'      253            3       20   
3499     3242      b'The Man With Nine Lives'      253            3       20   
3500     3244  b'Greetings from Earth, Pt. 1'      253            3       20   
3501     3224      b'Through a Looking Glass'      229            3       21   
3502     2820       b'Occupation / Precipice'      227            3       19   

            Composer  Milliseconds     

In [10]:
# 2.	Display the Customer ID and the total amount the customer has spent for customers that spent over $40.
B = dbGetQuery(con, "select customerid, sum(total) from Invoice group by CustomerId having sum(total) > 40;")
print(B)

    CustomerId  sum(total)
0            5       40.62
1            6       49.62
2            7       42.62
3           24       43.62
4           25       42.62
5           26       47.62
6           28       43.62
7           37       43.62
8           43       40.62
9           44       41.62
10          45       45.62
11          46       45.62
12          48       40.62
13          57       46.62


In [11]:
# 3.	Display the Artist ID for Queen.
C = dbGetQuery(con,"select artistid from Artist where Name = 'Queen' ")
print(C)

   ArtistId
0        51


In [12]:
# 4.	Display the 10 shortest songs.
D = dbGetQuery(con," select * from Track order by Milliseconds limit 10; ")
print(D)

   TrackId                            Name  AlbumId  MediaTypeId  GenreId  \
0     2461  b'\xc9 Uma Partida De Futebol'      200            1        1   
1      168                   b'Now Sports'       18            1        4   
2      170                  b'A Statistic'       18            1        4   
3      178                        b'Oprah'       18            1        4   
4     3304                 b'Commercial 1'      258            1       17   
5      172             b'The Real Problem'       18            1        4   
6     3310                 b'Commercial 2'      258            1       17   
7     2241                        b'Bossa'      184            1       17   
8     1086                b'Casinha Feliz'       85            1       10   
9      246                 b'Mateus Enter'       24            1        7   

           Composer  Milliseconds    Bytes  UnitPrice  
0    b'Samuel Rosa'          1071    38747       0.99  
1              None          4884   1612

In [13]:
# 5.	Display the track name and album title of all tracks that cost $1.99.
E = dbGetQuery(con,"select Name, Title from Track join Album using (AlbumId) where UnitPrice=1.99")
print(E)

                                          Name  \
0    b'Battlestar Galactica: The Story So Far'   
1                    b'Occupation / Precipice'   
2                             b'Exodus, Pt. 1'   
3                             b'Exodus, Pt. 2'   
4                             b'Collaborators'   
..                                         ...   
208       b"There's No Place Like Home, Pt. 1"   
209       b"There's No Place Like Home, Pt. 2"   
210       b"There's No Place Like Home, Pt. 3"   
211                          b'Branch Closing'   
212                              b'The Return'   

                                         Title  
0    b'Battlestar Galactica: The Story So Far'  
1            b'Battlestar Galactica, Season 3'  
2            b'Battlestar Galactica, Season 3'  
3            b'Battlestar Galactica, Season 3'  
4            b'Battlestar Galactica, Season 3'  
..                                         ...  
208                          b'LOST, Season 4'  
209    

In [14]:
# 6.	Display the artist name, album title, length of the album in seconds, with only albums longer than 1000 seconds, ordered by length in descending order.
F = dbGetQuery(con,"select Artist.Name, Title, sum(Milliseconds)/1000 as Length from Album join Artist using (ArtistId) join Track using (AlbumId) group by AlbumId having Length>1000 order by Length desc")
print(F)

                                  Name  \
0                              b'Lost'   
1    b'Battlestar Galactica (Classic)'   
2                              b'Lost'   
3                              b'Lost'   
4                            b'Heroes'   
..                                 ...   
251                    b'Led Zeppelin'   
252                       b'Van Halen'   
253                       b'Raimundos'   
254                        b'BackBeat'   
255             b'Black Label Society'   

                                           Title  Length  
0                              b'Lost, Season 3'   70665  
1    b'Battlestar Galactica (Classic), Season 1'   70213  
2                              b'Lost, Season 1'   64854  
3                              b'Lost, Season 2'   63289  
4                            b'Heroes, Season 1'   59780  
..                                           ...     ...  
251                                      b'Coda'    1981  
252                    

In [15]:
# 7.	Display the customer last name and first name, the track name, the unit price and
# the quantity for all track purchases.
G = dbGetQuery(con,"select customer.lastname,customer.firstname,InvoiceLine.unitprice,InvoiceLine.quantity,track.name from Customer join Invoice using(customerid) join InvoiceLine using(invoiceid) join Track using (trackid)")
print(G)

                    LastName    FirstName  UnitPrice  Quantity  \
0               b'K\xf6hler'    b'Leonie'       0.99         1   
1               b'K\xf6hler'    b'Leonie'       0.99         1   
2                  b'Hansen'  b'Bj\xf8rn'       0.99         1   
3                  b'Hansen'  b'Bj\xf8rn'       0.99         1   
4                  b'Hansen'  b'Bj\xf8rn'       0.99         1   
...                      ...          ...        ...       ...   
2235  b'H\xe4m\xe4l\xe4inen'     b'Terhi'       0.99         1   
2236  b'H\xe4m\xe4l\xe4inen'     b'Terhi'       0.99         1   
2237  b'H\xe4m\xe4l\xe4inen'     b'Terhi'       0.99         1   
2238  b'H\xe4m\xe4l\xe4inen'     b'Terhi'       0.99         1   
2239               b'Pareek'     b'Manoj'       1.99         1   

                                                   Name  
0                                  b'Balls to the Wall'  
1                                  b'Restless and Wild'  
2                              b'

In [16]:
# 8.	Display all customer information for all customers who had two or more orders of over $10.
H = dbGetQuery(con, "select distinct Customer.* from customer join Invoice as T1 using (customerid) join Invoice as T2 using (customerid) where T1.total >10 and  T2.total >10 and T1.invoiceid != T2.invoiceid ")
print(H)

   CustomerId   FirstName       LastName                   Company  \
0          37     b'Fynn'  b'Zimmermann'                      None   
1          34  b'Jo\xe3o'   b'Fernandes'                      None   
2          17     b'Jack'       b'Smith'  b'Microsoft Corporation'   
3          28    b'Julia'     b'Barnett'                      None   
4          57     b'Luis'       b'Rojas'                      None   

                       Address               City  State      Country  \
0       b'Berger Stra\xdfe 10'       b'Frankfurt'   None   b'Germany'   
1  b'Rua da Assun\xe7\xe3o 53'          b'Lisbon'   None  b'Portugal'   
2           b'1 Microsoft Way'         b'Redmond'  b'WA'       b'USA'   
3               b'302 S 700 E'  b'Salt Lake City'  b'UT'       b'USA'   
4           b'Calle Lira, 198'        b'Santiago'   None     b'Chile'   

      PostalCode                  Phone                   Fax  \
0       b'60316'    b'+49 069 40598889'                  None   
1         

In [17]:
I = dbGetQuery(con, """
    SELECT Track.Name, SUM(InvoiceLine.Quantity) as TotalQuantity
    FROM Track
    JOIN InvoiceLine ON Track.TrackId = InvoiceLine.TrackId
    GROUP BY Track.TrackId
    ORDER BY TotalQuantity DESC
    LIMIT 5;
""")
print(I)


                   Name  TotalQuantity
0  b'Balls to the Wall'              2
1   b'Inject The Venom'              2
2         b'Snowballed'              2
3           b'Overdose'              2
4    b'Deuces Are Wild'              2
